# Job Scheduling Problem

We have fifteen jobs, each with its processing time, that need to be scheduled on three machines. Certain jobs are conflicting, meaning they cannot be scheduled on the same machine. The table below lists the job IDs, processing times, and sets of conflicting jobs:

| Job | Processing Time | Conflicting Jobs |
|-----|-----------------|------------------|
| 1   | 7               | None             |
| 2   | 4               | 5, 8             |
| 3   | 6               | None             |
| 4   | 9               | None             |
| 5   | 12              | 2, 8             |
| 6   | 8               | 9                |
| 7   | 10              | 10               |
| 8   | 11              | 2, 5             |
| 9   | 8               | 6                |
| 10  | 7               | 7                |
| 11  | 6               | 15               |
| 12  | 8               | None             |
| 13  | 15              | None             |
| 14  | 14              | None             |
| 15  | 3               | 11               |

The goal is to schedule the jobs to minimize the makespan (the maximum completion time among the machines). For example, one possible schedule could be:
- Machine 1: Jobs 1, 4, 7, 8, and 13
- Machine 2: Jobs 2, 6, 10, 11, and 14
- Machine 3: Jobs 3, 5, 9, 12, and 15

In this schedule, the total processing times on the three machines are 52, 39, and 37, respectively, resulting in a makespan of 52. While this is feasible, it may not be optimal. When trying to improve the schedule, we need to consider conflicting jobs. For instance, exchanging jobs 8 and 11 to reduce the makespan would result in machine 2 processing conflicting jobs 2 and 8, which is not feasible.


In [20]:
from gurobipy import *
import pandas as pd  

In [21]:
problem1 = pd.read_excel('OR2_06_quiz_data .xlsx', 'Problem 1')

In [22]:
problem1.drop(columns=problem1.columns[-3:], inplace=True)
problem1['Conflicting jobs'] = problem1['Conflicting jobs'].apply(lambda x: list(map(int, x.split(', '))) if x != "None" else [])

In [23]:
machines = list(range(1, 4))
print(problem1)

    Job  Processing time Conflicting jobs
0     1                7               []
1     2                4           [5, 8]
2     3                6               []
3     4                9               []
4     5               12           [2, 8]
5     6                8              [9]
6     7               10             [10]
7     8               11           [2, 5]
8     9                8              [6]
9    10                7              [7]
10   11                6             [15]
11   12                8               []
12   13               15               []
13   14               14               []
14   15                3             [11]


In [24]:
jobs = problem1['Job']
processing_time = problem1['Processing time']
conflicting_jobs = problem1['Conflicting jobs']

In [25]:
model_1 = Model("Job Scheduling")

In [26]:
#creating the variables using addvars
x = model_1.addVars(machines, jobs, vtype = GRB.BINARY, name = "x")

In [27]:
#showing the variables
model_1.update()
model_1.getVars()
#showing the number of variable created
print("Number of variables created: ", model_1.NumVars)

Number of variables created:  45


In [28]:
model_1.getVars()

[<gurobi.Var x[1,1]>,
 <gurobi.Var x[1,2]>,
 <gurobi.Var x[1,3]>,
 <gurobi.Var x[1,4]>,
 <gurobi.Var x[1,5]>,
 <gurobi.Var x[1,6]>,
 <gurobi.Var x[1,7]>,
 <gurobi.Var x[1,8]>,
 <gurobi.Var x[1,9]>,
 <gurobi.Var x[1,10]>,
 <gurobi.Var x[1,11]>,
 <gurobi.Var x[1,12]>,
 <gurobi.Var x[1,13]>,
 <gurobi.Var x[1,14]>,
 <gurobi.Var x[1,15]>,
 <gurobi.Var x[2,1]>,
 <gurobi.Var x[2,2]>,
 <gurobi.Var x[2,3]>,
 <gurobi.Var x[2,4]>,
 <gurobi.Var x[2,5]>,
 <gurobi.Var x[2,6]>,
 <gurobi.Var x[2,7]>,
 <gurobi.Var x[2,8]>,
 <gurobi.Var x[2,9]>,
 <gurobi.Var x[2,10]>,
 <gurobi.Var x[2,11]>,
 <gurobi.Var x[2,12]>,
 <gurobi.Var x[2,13]>,
 <gurobi.Var x[2,14]>,
 <gurobi.Var x[2,15]>,
 <gurobi.Var x[3,1]>,
 <gurobi.Var x[3,2]>,
 <gurobi.Var x[3,3]>,
 <gurobi.Var x[3,4]>,
 <gurobi.Var x[3,5]>,
 <gurobi.Var x[3,6]>,
 <gurobi.Var x[3,7]>,
 <gurobi.Var x[3,8]>,
 <gurobi.Var x[3,9]>,
 <gurobi.Var x[3,10]>,
 <gurobi.Var x[3,11]>,
 <gurobi.Var x[3,12]>,
 <gurobi.Var x[3,13]>,
 <gurobi.Var x[3,14]>,
 <gurobi.Var x[

In [35]:
makespan = model_1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "makespan")

In [36]:
#setting objective function
model_1.setObjective(makespan, GRB.MINIMIZE)

In [37]:
#constraint for performing each job once
model_1.addConstrs((quicksum(x[i, j] for i in machines) == 1 for j in jobs))
#constraint for not performing conflicting jobs on the same machine
model_1.addConstrs((quicksum(x[i, g] for g in conflicting_jobs[j-1]) <= 1 for i in machines for j in jobs))

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 10): <gurobi.Constr *Awaiting Model Update*>,
 (1, 11): <gurobi.Constr *Awaiting Model Update*>,
 (1, 12): <gurobi.Constr *Awaiting Model Update*>,
 (1, 13): <gurobi.Constr *Awaiting Model Update*>,
 (1, 14): <gurobi.Constr *Awaiting Model Update*>,
 (1, 15): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Upda

In [39]:
model_1.addConstrs((makespan >= quicksum(processing_time[j-1]*x[i, j] for j in jobs) for i in machines))

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [40]:
model_1.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 63 rows, 47 columns and 129 nonzeros
Model fingerprint: 0xad127feb
Variable types: 2 continuous, 45 integer (45 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 75.0000000
Presolve removed 42 rows and 1 columns
Presolve time: 0.04s
Presolved: 21 rows, 46 columns, 102 nonzeros
Variable types: 0 continuous, 46 integer (45 binary)

Root relaxation: objective 4.266667e+01, 25 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   42.66667    0    

In [42]:
print("Result:")

for i in machines:
    print("Jobs on machine " + str(i) + ": ", end=" ")
    for j in jobs:
        if x[i,j].x == 1:
            print(str(j) + " ", end=" ")
            print()
print("makespan = ", model_1.objVal)             

Result:
Jobs on machine 1:  1  
4  
5  
14  
Jobs on machine 2:  2  
6  
7  
11  
13  
Jobs on machine 3:  3  
8  
9  
10  
12  
15  
makespan =  43.0


In [43]:
#writing the model to a file
model_1.write("JobScheduling.lp")

In [44]:
#writing the solution to a file
model_1.write("JobScheduling.sol")